In [4]:
import os
import rioxarray
import xarray as xr
import geopandas as gpd
import pandas as pd
import numpy as np
import os.path
from pathlib import Path
import csv
import math
from osgeo import gdal, ogr, osr, gdalconst


here's a starting point comparison https://github.com/chudasama-bijal/QGIS-Plugin-Weights-of-Evidence-Model

Get training data vector data - 1s

Choose mask layer if desired

Classes
- Favourable
- Unfavourable

Get list of evidence rasters

Studentized t contrast

Clip and make rasters the same details

Rasterise training data

Get count of data and not - e.g 1s and 0s in raster

Calculate weights
#633

Input weights rasters